Collecting the dataset

In [4]:
import cv2

def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    
    def face_cropped(img):
        if img is None:
            return None
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        
        if len(faces) == 0:
            return None
        
        # Crop only the last detected face
        x, y, w, h = faces[-1]
        cropped_face = img[y:y+h, x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture(0)  # Change camera index to 0 for default camera
    
    max_images_per_user = 1000  # Maximum number of images to capture per user
    id = 1
    img_id = 0
    
    while True:
        ret, frame = cap.read()
        
        if not ret:
            continue  # If frame capture fails, try again
        
        cropped_face = face_cropped(frame)
        
        if cropped_face is not None:
            img_id += 1
            face = cv2.resize(cropped_face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = f"data/user.{id}.{img_id}.jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped face", face)
        
        if cv2.waitKey(100) == 13 or img_id == max_images_per_user:
            id += 1  # Increment id for the next user
            img_id = 0  # Reset img_id for the next user
            if img_id == max_images_per_user:
                print(f"Maximum images per user ({max_images_per_user}) reached. Moving to the next user.")
            if cv2.waitKey(0) == 13:  # Wait for Enter key to continue
                break
    
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")

generate_dataset()

KeyboardInterrupt: 

In [3]:
import cv2
import os
import numpy as np
from PIL import Image
import face_recognition

# Load the dataset
dataset_path = "data/"
face_data = []
labels = []
class_id = 0
names = {}  # Mapping class_id to names

# Prepend the images from the dataset
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith("jpg"):
            path = os.path.join(root, file)
            label = os.path.basename(path).split(".")[0]  # Extract the label from the filename
            
            # Assign a name to the label
            if label not in names:
                name = input(f"Enter the name for label '{label}': ")
                names[label] = class_id
                class_id += 1
            
            id_ = names[label]  # Get the class_id from the label
            
            pil_image = Image.open(path).convert("RGB")  # Convert to RGB
            image_array = np.array(pil_image, "uint8")
            face_data.append(image_array)
            labels.append(id_)

# Train the face recognition model
model = []
for image in face_data:
    face_encodings = face_recognition.face_encodings(image)
    if len(face_encodings) > 0:
        face_encoding = face_encodings[0]
        model.append(face_encoding)
    else:
        print("No face detected in one of the images. Skipping...")

model = np.array(model)

# Save the trained model
np.save("trained_model.npy", model)
np.save("labels.npy", labels)
np.save("names.npy", names)

print("Training completed successfully.")

Enter the name for label 'user':  Adipta


No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the images. Skipping...
No face detected in one of the 

In [ ]:
import numpy as np
import cv2
import face_recognition

# Load the trained model, labels, and names
model = np.load("trained_model.npy")
labels = np.load("labels.npy")
names = np.load("names.npy", allow_pickle=True).item()

# Initialize the webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the frame to RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces in the frame
    face_locations = face_recognition.face_locations(frame_rgb)
    face_encodings = face_recognition.face_encodings(frame_rgb, face_locations)

    # Match face encodings with the trained model
    for face_encoding, face_location in zip(face_encodings, face_locations):
        matches = face_recognition.compare_faces(model, face_encoding)
        name = "Unknown"
        matching_labels = [labels[i] for i, match in enumerate(matches) if match]
        if matching_labels:
            names_found = [names[label] for label in matching_labels if label in names]
            if names_found:
                name = "shubham".join(names_found)

        # Draw rectangle around the detected face and label it
        top, right, bottom, left = face_location
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display the frame
    cv2.imshow('Face Recognition', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


In [6]:
import numpy as np
import cv2
import face_recognition

# Load the trained model, labels, and names
model = np.load("trained_model.npy")
labels = np.load("labels.npy")
names = np.load("names.npy", allow_pickle=True).item()

# Initialize the webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the frame to RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces in the frame
    face_locations = face_recognition.face_locations(frame_rgb)
    face_encodings = face_recognition.face_encodings(frame_rgb, face_locations)

    # Match face encodings with the trained model
    for face_encoding, face_location in zip(face_encodings, face_locations):
        matches = face_recognition.compare_faces(model, face_encoding)
        name = "Unknown"
        matching_labels = [labels[i] for i, match in enumerate(matches) if match]
        print("Matching labels:", matching_labels)  # Debugging
        if matching_labels:
            names_found = [names[label] for label in matching_labels if label in names]
            print("Names found:", names_found)  # Debugging
            if names_found:
                name = ", ".join(names_found)

        # Draw rectangle around the detected face and label it
        top, right, bottom, left = face_location
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display the frame
    cv2.imshow('Face Recognition', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


Matching labels: []
Matching labels: []
Matching labels: [0, 0]
Names found: []
Matching labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Names found: []
Matching labels: [0]
Names found: []
Matching labels: [0]
Names fou

KeyboardInterrupt: 

In [3]:
import numpy as np

# Load the .npy file with allow_pickle=True
data = np.load("labels.npy", allow_pickle=True)

# Now you can work with the loaded data
print(data)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [1]:
import cv2
import numpy as np
import face_recognition

# Load the trained model
model = np.load("trained_model.npy")
labels = np.load("labels.npy")
names = np.load("names.npy", allow_pickle=True).item()  # Load names as a dictionary

# Initialize the webcam
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # Convert the image from BGR color (which OpenCV uses) to RGB color
    rgb_frame = frame[:, :, ::-1]

    # Find all face locations and encodings in the current frame
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Loop through each face found in the frame
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Compare face encodings with the trained model to recognize the face
        matches = face_recognition.compare_faces(model, face_encoding)
        name = "Unknown"

        # Check if we have a match
        if True in matches:
            matched_index = np.argmax(matches)
            label = str(labels[matched_index])  # Convert label to string
            name = names.get(label, "Unknown")  # Use get method to handle KeyError

        # Draw a rectangle around the face and label it
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # If 'q' is pressed, exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close any open windows
video_capture.release()
cv2.destroyAllWindows()
